In [1]:
! pip install scipy --quiet
! pip install tenacity --quiet
! pip install tiktoken --quiet
! pip install termcolor --quiet
! pip install openai --quiet

In [2]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-4o"
client = OpenAI()

In [3]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [5]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='Could you please specify the city and state for which you would like to know the weather?', role='assistant', function_call=None, tool_calls=None)

In [8]:
def heapify(arr, n, i):
    largest = i
    l = 2 * i + 1
    r = 2 * i + 2

    if l < n and arr[i] < arr[l]:
        largest = l

    if r < n and arr[largest] < arr[r]:
        largest = r

    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        print(arr)
        heapify(arr, n, largest)

def heap_sort(arr):
    n = len(arr)
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)

    for i in range(n-1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]
        heapify(arr, i, 0)
    return arr

l = [100, 3, 93, 81, 74, 54, 65, 51, 43, 35, 28, 16, 75, 48, 20, 17, 55, 99, 62]
print("Heap Sort:", heap_sort(l.copy()))


[100, 3, 93, 81, 74, 54, 65, 51, 99, 35, 28, 16, 75, 48, 20, 17, 55, 43, 62]
[100, 3, 93, 81, 74, 54, 65, 55, 99, 35, 28, 16, 75, 48, 20, 17, 51, 43, 62]
[100, 3, 93, 81, 74, 75, 65, 55, 99, 35, 28, 16, 54, 48, 20, 17, 51, 43, 62]
[100, 3, 93, 99, 74, 75, 65, 55, 81, 35, 28, 16, 54, 48, 20, 17, 51, 43, 62]
[100, 99, 93, 3, 74, 75, 65, 55, 81, 35, 28, 16, 54, 48, 20, 17, 51, 43, 62]
[100, 99, 93, 81, 74, 75, 65, 55, 3, 35, 28, 16, 54, 48, 20, 17, 51, 43, 62]
[100, 99, 93, 81, 74, 75, 65, 55, 62, 35, 28, 16, 54, 48, 20, 17, 51, 43, 3]
[99, 3, 93, 81, 74, 75, 65, 55, 62, 35, 28, 16, 54, 48, 20, 17, 51, 43, 100]
[99, 81, 93, 3, 74, 75, 65, 55, 62, 35, 28, 16, 54, 48, 20, 17, 51, 43, 100]
[99, 81, 93, 62, 74, 75, 65, 55, 3, 35, 28, 16, 54, 48, 20, 17, 51, 43, 100]
[99, 81, 93, 62, 74, 75, 65, 55, 43, 35, 28, 16, 54, 48, 20, 17, 51, 3, 100]
[93, 81, 3, 62, 74, 75, 65, 55, 43, 35, 28, 16, 54, 48, 20, 17, 51, 99, 100]
[93, 81, 75, 62, 74, 3, 65, 55, 43, 35, 28, 16, 54, 48, 20, 17, 51, 99, 100]